# Data Cleaning and Preparation

In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

In [3]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [10]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [11]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [12]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by refer‐ ring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example).

When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA in object arrays:

In [15]:
string_data[0] = None

In [14]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

There is work ongoing in the pandas project to improve the internal details of how missing data is handled, but the user API functions, like pandas.isnull, abstract away many of the annoying details. 

See Table 7-1 for a list of some functions related to missing data handling.

Table 7-1. NA handling methods. PAGE 192

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using **pandas.isnull** and boolean indexing, the **dropna** can be helpful. On a Series, it returns the Series with only the non-null data and index values:


In [16]:
from numpy import nan as NA

In [212]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [213]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [215]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. **dropna by default drops any row containing a missing value:**


In [17]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [18]:
cleaned = data.dropna()

In [19]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [20]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:

In [21]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [27]:
data[4] = NA #added a column

In [26]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


To drop columns in the same way, pass axis=1:

In [25]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [217]:
df = pd.DataFrame(np.random.randn(7, 3))

In [218]:
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810
3,-0.713544,-0.831154,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


In [219]:
df.iloc[:4, 1] = NA #COLUMNS THEN ROW

In [30]:
df.iloc[:2, 2] = NA

In [32]:
df

,0,1,2
0,0.476985,NaN,NaN
1,-0.577087,NaN,NaN
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


In [33]:
df.dropna()

,0,1,2
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the **thresh argument**:

In [34]:
df.dropna(thresh=2)

,0,1,2
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


### Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the **fillna** method is the workhorse function to use. Calling **fillna** with a constant replaces missing values with that value:

In [220]:
df

,0,1,2
0,0.476985,NaN,-1.021228
1,-0.577087,NaN,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


In [35]:
df.fillna(0)

,0,1,2
0,0.476985,0.000000,0.000000
1,-0.577087,0.000000,0.000000
2,0.523772,0.000000,1.343810
3,-0.713544,0.000000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


Calling fillna with a dict, you can use a different fill value for each column:

In [36]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.476985,0.500000,0.000000
1,-0.577087,0.500000,0.000000
2,0.523772,0.500000,1.343810
3,-0.713544,0.500000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


fillna returns a new object, but you can modify the existing object in-place:

In [221]:
_ = df.fillna(0, inplace=True) #inplace provides modification of the existing object
df

,0,1,2
0,0.476985,0.000000,-1.021228
1,-0.577087,0.000000,0.302614
2,0.523772,0.000000,1.343810
3,-0.713544,0.000000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


The same interpolation methods available for reindexing can be used with fillna:

In [227]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,1.349742,0.069877
3,0.246674,-0.011862,1.004812
4,1.327195,-0.919262,-1.549106
5,0.022185,0.758363,-0.660524


In [229]:
df.iloc[2:, 1] = NA #[index, column]
df

,0,1,2
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,NaN,0.069877
3,0.246674,NaN,1.004812
4,1.327195,NaN,-1.549106
5,0.022185,NaN,-0.660524


In [226]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,0.0,-1.021228
1,-0.577087,0.0,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN
6,0.332883,NaN,NaN


In [42]:
df.fillna(method='ffill') #ffill takes the value on top and fills the rest of the way down. praprogation

,0,1,2
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,0.377984,0.069877
3,0.246674,0.377984,1.004812
4,1.327195,0.377984,1.004812
5,0.022185,0.377984,1.004812


In [41]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,0.377984,0.069877
3,0.246674,0.377984,1.004812
4,1.327195,NaN,1.004812
5,0.022185,NaN,1.004812



With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [43]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [44]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

See Table 7-2 for a reference on fillna. PAGE 196

## Data Transformation

### Removing Duplicates


Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [230]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [231]:
data.duplicated() #compare to the Previous row

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, **drop_duplicates** returns a DataFrame where the duplicated array is False:

In [234]:
data.drop_duplicates() #6 was removed

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [245]:
data['v1'] = range(7)

In [246]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [247]:
data.drop_duplicates(['k1']) #dropped all the duplicated in 'k1'

,k1,k2,v1
0,one,1,0
1,two,1,1



duplicated and drop_duplicates by default keep the first observed value combination. Passing keep='last' will return the last one:

In [58]:
data.drop_duplicates(['k1', 'k2'], keep='last') #kept #6 when #5 and #6 were duplicates

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [59]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [60]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

The **map** method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the str.lower Series method:

In [61]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [64]:
data['animal'] = lowercased.map(meat_to_animal) #adding an animal column

data #The map function helps us iteratively apply a function to all the items in a Python list, or in any Python iterable for that matter, with just one line of code.

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [65]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) ##REVIEW THIS

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

### Replacing Values

Filling in missing data with the **fillna** method is a special case of more general value replacement. As you’ve already seen, map can be used to modify a subset of values in an object but **replace** provides a simpler and more flexible way to do so. Let’s consider this Series:

In [66]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use **replace**, producing a new Series (unless you pass inplace=True):

In [67]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value:

In [68]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [69]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [70]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The data.replace method is distinct from data.str.replace, which performs string substitution element-wise. We look at these string methods on Series later in the chapter.

### Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [90]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

Like a Series, the axis indexes have a map method:

In [91]:
transform = lambda x: x[:4].upper() #4 characters
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [92]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is rename:

In [93]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11



Notably, **rename** can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [94]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


**rename** saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:

In [95]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [96]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use cut, a function in pandas:

In [97]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by **pandas.cut**. You can treat it like an array of strings indicating the bin name; internally it contains a **categories** array specifying the distinct category names along with a labeling for the ages data in the **codes** attribute:

In [103]:
cats.codes #12 integars for 12 different bins and their order in an series

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [99]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [102]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Consistent with mathematical notation for intervals, a parenthesis means that the side is **open**, while the square bracket means it is **closed** (inclusive). You can change which side is closed by passing **right=False**:

In [248]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) #open means it includes up to that number. closed means its between that number.

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [105]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [107]:
data = np.random.rand(20) ##REVIEW THIS **
pd.cut(data, 4, precision=2)

[(0.72, 0.96], (0.02, 0.26], (0.02, 0.26], (0.26, 0.49], (0.49, 0.72], ..., (0.26, 0.49], (0.02, 0.26], (0.72, 0.96], (0.49, 0.72], (0.49, 0.72]]
Length: 20
Categories (4, interval[float64, right]): [(0.02, 0.26] < (0.26, 0.49] < (0.49, 0.72] < (0.72, 0.96]]

**The precision=2 option limits the decimal precision to two digits**

A closely related function, **qcut**, bins the data based on sample quantiles. Depending on the distribution of the data, using **cut** will not usually result in each bin having the same number of data points. Since **qcut** uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [108]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-0.686, -0.0453], (0.604, 3.928], (-0.0453, 0.604], (0.604, 3.928], (-0.0453, 0.604], ..., (-0.0453, 0.604], (-2.9499999999999997, -0.686], (-0.0453, 0.604], (0.604, 3.928], (-0.686, -0.0453]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.686] < (-0.686, -0.0453] < (-0.0453, 0.604] < (0.604, 3.928]]

In [109]:
pd.value_counts(cats)

(-2.9499999999999997, -0.686]    250
(-0.686, -0.0453]                250
(-0.0453, 0.604]                 250
(0.604, 3.928]                   250
dtype: int64

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [110]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.191, -0.0453], (1.287, 3.928], (-0.0453, 1.287], (-0.0453, 1.287], (-0.0453, 1.287], ..., (-0.0453, 1.287], (-2.9499999999999997, -1.191], (-0.0453, 1.287], (-0.0453, 1.287], (-1.191, -0.0453]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -1.191] < (-1.191, -0.0453] < (-0.0453, 1.287] < (1.287, 3.928]]


We’ll return to cut and qcut later in the chapter during our discussion of aggregation and group operations, as these discretization functions are especially useful for quan‐ tile and group analysis.

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [4]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [5]:
col = data[2]
col

0     -0.519439
1      0.092908
2      1.007189
3      1.352917
4      1.669025
         ...   
995   -1.451985
996   -1.060881
997    0.037333
998   -0.782960
999    1.177706
Name: 2, Length: 1000, dtype: float64

In [6]:
col[np.abs(col) > 3]

5      3.248944
102    3.176873
324    3.260383
499   -3.056990
586   -3.184377
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame:

In [7]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


Values can be set based on these criteria. Here is code to cap values outside the inter‐ val –3 to 3:

In [8]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


The statement np.sign(data) produces 1 and –1 values based on whether the values in data are positive or negative:

In [9]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,1.0,1.0
4,-1.0,-1.0,1.0,-1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the **numpy.random.permutation** function. Calling **permutation** with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [10]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [11]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

That array can then be used in iloc-based indexing or the equivalent take function:

In [12]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [13]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


To select a random subset without replacement, you can use the sample method on Series and DataFrame:

In [14]:
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


To generate a sample with replacement (to allow repeat choices), pass **replace=True** to sample:

In [15]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws##**REVIEW**

4    4
4    4
2   -1
2   -1
2   -1
0    5
3    6
0    5
4    4
1    7
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or Data‐ Frame with k columns containing all 1s and 0s. pandas has a **get_dummies** function for doing this, though devising one yourself is not difficult. Let’s return to an earlier example DataFrame:

In [127]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. **get_dummies** has a prefix argument for doing this:

In [128]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more complicated. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Chapter 14:

In [131]:
import fileinput

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies

/var/folders/b8/qw7xf4r16w383xrhw3980l4w0000gp/T/ipykernel_57517/761451698.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [132]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, we extract the list of unique genres in the dataset:

In [133]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

Now we have:

In [134]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:

In [135]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

Now, iterate through each movie and set entries in each row of dummies to 1. To do this, we use the dummies.columns to compute the column indices for each genre:

In [137]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [138]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

Then, we can use .iloc to set values based on these indices:

In [144]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

Then, as before, you can combine this with movies:

In [145]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

A useful recipe for statistical applications is to combine get_dummies with a discreti‐ zation function like cut:

In [146]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [143]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


We set the random seed with numpy.random.seed to make the example deterministic. We will look again at pandas.get_dummies later in the book.

## String Manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

### String Object Methods

In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with **split**:

In [160]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

**split** is often combined with strip to **trim whitespace** (including line breaks):

In [161]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [162]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the **join** method on the string '::':

In [163]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is
the best way to detect a substring, though **index** and **find** can also be used:

In [164]:
'guido' in val

True

In [165]:
val.index(',')

1

In [166]:
val.find(':')

-1

Note the difference between find and index is that index raises an exception if the
string isn’t found (versus returning –1):

In [167]:
val.index(':')

ValueError: substring not found

Relatedly, **count** returns the number of occurrences of a particular substring:

In [168]:
val.count(',')

2

**replace** will substitute occurrences of one pattern for another. It is commonly used
to delete patterns, too, by passing an empty string:

In [170]:
val.replace(',', '::')

'a::b::  guido'

In [171]:
val.replace(',', '')

'ab  guido'

### Regular Expressions

Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a regex, is a string formed according to the regular expression language. Python’s built-in re module is responsible for applying regular expressions to strings; I’ll give a number of examples of its use here.

The re module functions fall into three categories: pattern matching, substitution, and splitting. Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example:

In [172]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

When you call re.split('\s+', text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with re.compile, forming a reusable regex object:


In [173]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the
findall method:

In [177]:
regex.findall(text)

['    ', '\t ', '  \t']

Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.

**match** and **search** are closely related to **findall**. While findall returns all matches in a string, search returns only the first match. More rigidly, match only matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [181]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [179]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

**search** returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [182]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [183]:
text[m.start():m.end()]

'dave@google.com'

**regex.match** returns **None**, as it only will match if the pattern occurs at the start of the
string:

In [184]:
print(regex.match(text))

None



Relatedly, sub will return a new string with occurrences of the pattern replaced by the a new string:

In [185]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [186]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern com‐ ponents with its groups method:

In [187]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

findall returns a list of tuples when the pattern has groups:

In [188]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [189]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



There is much more to regular expressions in Python, most of which is outside the
book’s scope. Table 7-4 provides a brief summary.

### Vectorized String Functions in pandas

In [190]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

In [191]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [192]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

You can apply string and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string opera‐ tions that skip NA values. These are accessed through Series’s str attribute; for exam‐ ple, we could check whether each email address has 'gmail' in it with str.contains:

In [203]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any re options like IGNORECASE:

In [195]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [205]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use str.get or index into the str attribute:

In [206]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

To access elements in the embedded lists, we can pass an index to either of these functions:

In [201]:
matches.str.get(1)

AttributeError: Can only use .str accessor with string values!

In [202]:
matches.str[0]

AttributeError: Can only use .str accessor with string values!

You can similarly slice strings using this syntax:

In [207]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [210]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

See Table 7-5 for more pandas string methods. PAGE 218

## Conclusion